### OpenSearch document ingestion pipeline

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import sys
import json
if "../../src" not in sys.path:
    sys.path.append("../../src")
import utils
import settings
from opensearch_client import OpenSearchClient
from pdf_processor import PdfProcessor

wrapper = utils.get_text_wrapper()

In [14]:
INDEX_NAME = "unstructured-knn-index"

Get some pdfs

In [15]:
pdfs = [
    {
        "file_path": settings.DATA_DIR / "admitere" / "Test limba straina 21 03 detalii S1 2025.pdf",
        "url": "https://admitere.pub.ro/docs/Test%20limba%20straina%2021%2003%20detalii%20S1%202025.pdf",
        "output_dir": settings.DATA_DIR / "content" / "admitere",
    }
]

pdf = pdfs[0]
file_path, url = pdf["file_path"], pdf["url"]

Process pdf

In [16]:
pdfProcessor = PdfProcessor(file_path, url=url)
pdfProcessor.process()

In [17]:
for chunk in pdfProcessor.chunks:
    print(wrapper.fill(json.dumps(chunk, indent=4, ensure_ascii=False)))
    print(wrapper.fill("-" * wrapper.width))

{
    "id": "9cf23be3e6b1bcbd56f0cab3918a505c41688bb641a64cac3a8509bc9628e5d1-1",
    "text": "Despre testul de limbă
on-line POLITEHNICA București din data de 21 martie 2025 L Ce este cu acest test? Am nevoie de el? Dacă doresc să
beneficiez de avantajele studiului într-una din limbile de largă circulație (engleză, franceză, germană) și în lista mea
de opțiuni includ domenii la care școlarizarea se desfășoară într-una din acest limbi, trebuie, conform legii, să fac
dovada că am cunoștințele lingvistice necesare. Dovada se face folosind oricare dintre variantele de mai jos:",
"url": "https://admitere.pub.ro/docs/Test%20limba%20straina%2021%2003%20detalii%20S1%202025.pdf",
    "type": "pdf",
"filename": "Test limba straina 21 03 detalii S1 2025.pdf",
    "page_number": 1,
    "table_id": null,
"table_text": null
}
------------------------------------------------------------------------------------------------------------------------
{
    "id": "9cf23be3e6b1bcbd56f0cab3918a505c41688bb64

Ingest data into OpenSearch

In [18]:
data = pdfProcessor.format_data(index_name=INDEX_NAME)

In [19]:
client = OpenSearchClient()

In [20]:
ret = client.ingest_data_bulk(data)

In [21]:
for obj in ret:
    print(json.dumps(obj, indent=4))

[
    true,
    {
        "index": {
            "_index": "unstructured-knn-index",
            "_id": "9cf23be3e6b1bcbd56f0cab3918a505c41688bb641a64cac3a8509bc9628e5d1-1",
            "_version": 11,
            "result": "updated",
            "_shards": {
                "total": 2,
                "successful": 2,
                "failed": 0
            },
            "_seq_no": 32,
            "_primary_term": 3,
            "status": 200
        }
    }
]
[
    true,
    {
        "index": {
            "_index": "unstructured-knn-index",
            "_id": "9cf23be3e6b1bcbd56f0cab3918a505c41688bb641a64cac3a8509bc9628e5d1-2",
            "_version": 9,
            "result": "updated",
            "_shards": {
                "total": 2,
                "successful": 2,
                "failed": 0
            },
            "_seq_no": 33,
            "_primary_term": 3,
            "status": 200
        }
    }
]
[
    true,
    {
        "index": {
            "_index": "unstruc